In [64]:
#!pip install yfinance
#!pip install ipynb

In [65]:
#!pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt


In [66]:
stock = input()
df= yf.download(stock, '2020-10-18', '2022-10-20') 


GS
[*********************100%***********************]  1 of 1 completed


In [67]:
df.shape[0]

505

In [68]:


df.reset_index( inplace = True)

df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-10-19,207.389999,207.910004,204.589996,205.690002,197.162338,3235300
1,2020-10-20,206.699997,211.380005,206.470001,208.029999,199.405334,3335500
2,2020-10-21,207.389999,207.949997,202.009995,202.910004,194.497589,3083600
3,2020-10-22,203.199997,206.369995,202.199997,205.399994,196.884369,2136900
4,2020-10-23,206.759995,208.210007,204.020004,205.039993,196.539276,2110200


In [69]:
def support(df, index, n1, n2):
    # n1 (n2) candle before (after) the index candle resp.
    # test whether the index candle is support or not
    for i in range(index - n1 + 1, index + 1):
        if( df['Low'][i] > df['Low'][i-1]):
            return False
    for i in range(index+1, index + n2 + 1):
        if( df['Low'][i] < df['Low'][i-1] ):
            return False
    
    return True

In [70]:
def resistance(df, index, n1, n2):
    for i in range(index-n1+1, index+1):
        if(df['High'][i] < df['High'][i-1]):
            return False
    for i in range(index + 1, index + n2 + 1):
        if(df['High'][i] > df['High'][i-1]):
            return False
    
    return True


In [71]:

# using two list support_list, resistance_list

support_list = []
resistance_list = []
# add a volume and  time weighing
volsums=0
volsumr=0
vollists=[]
vollistr=[]
timelists=[]
timelistr=[]


n1 = 3
n2 = 2

for row in range(n1, len(df) - n2):
    if support(df, row, n1, n2 )&((max(df.index)-365)<=row):
        support_list.append((row , df['Low'][row], 0, df['Volume'][row], df['Date'][row]))
        volsums+=df['Volume'][row]
        timelists.append(df['Date'][row])
        vollists.append(df['Volume'][row])
    if resistance(df , row, n1, n2)&((max(df.index)-365)<=row):
        resistance_list.append((row, df['High'][row],1, df['Volume'][row], df['Date'][row]))
        volsumr+=df['Volume'][row]
        timelistr.append(df['Date'][row])
        vollistr.append(df['Volume'][row])
print(support_list)
print(len(support_list))
print(resistance_list)
print(len(resistance_list))
print(volsums)
print(volsumr)
vollists
vollistr
timelists
timelistr

[(142, 353.25, 0, 2673900, Timestamp('2021-05-13 00:00:00')), (147, 353.3500061035156, 0, 1919600, Timestamp('2021-05-20 00:00:00')), (167, 348.1300048828125, 0, 6290100, Timestamp('2021-06-18 00:00:00')), (180, 355.5899963378906, 0, 2940500, Timestamp('2021-07-08 00:00:00')), (187, 349.0, 0, 4394600, Timestamp('2021-07-19 00:00:00')), (210, 390.9100036621094, 0, 2241500, Timestamp('2021-08-19 00:00:00')), (231, 372.5, 0, 3834500, Timestamp('2021-09-20 00:00:00')), (241, 372.82000732421875, 0, 2895600, Timestamp('2021-10-04 00:00:00')), (248, 378.6300048828125, 0, 2391600, Timestamp('2021-10-13 00:00:00')), (264, 403.0299987792969, 0, 3802300, Timestamp('2021-11-04 00:00:00')), (275, 381.0400085449219, 0, 2610400, Timestamp('2021-11-19 00:00:00')), (283, 376.0299987792969, 0, 3343700, Timestamp('2021-12-02 00:00:00')), (318, 326.2300109863281, 0, 6569800, Timestamp('2022-01-24 00:00:00')), (370, 308.20001220703125, 0, 3562600, Timestamp('2022-04-07 00:00:00')), (415, 278.32000732421875

[Timestamp('2021-05-10 00:00:00'),
 Timestamp('2021-06-07 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-13 00:00:00'),
 Timestamp('2021-08-12 00:00:00'),
 Timestamp('2021-10-18 00:00:00'),
 Timestamp('2021-10-26 00:00:00'),
 Timestamp('2021-11-02 00:00:00'),
 Timestamp('2021-12-07 00:00:00'),
 Timestamp('2022-01-05 00:00:00'),
 Timestamp('2022-02-10 00:00:00'),
 Timestamp('2022-03-18 00:00:00'),
 Timestamp('2022-04-21 00:00:00'),
 Timestamp('2022-05-31 00:00:00'),
 Timestamp('2022-06-28 00:00:00'),
 Timestamp('2022-07-22 00:00:00'),
 Timestamp('2022-07-29 00:00:00'),
 Timestamp('2022-08-26 00:00:00'),
 Timestamp('2022-09-12 00:00:00'),
 Timestamp('2022-10-04 00:00:00')]

In [72]:
# support weights
volws=[]
timews=[]
target=vollists
target=pd.Series(target)
ttt=pd.to_datetime(timelists)
a=target.ewm(halflife='252 days', times=ttt).mean()

# a=target.ewm(alpha=0.95).mean()

for i in range(len(support_list)):
  vw=a[i]/sum(a)
  volws.append(vw)
volws

[0.04408599380096468,
 0.037807861935856626,
 0.06110197128982233,
 0.057670301419839275,
 0.060907102134389005,
 0.056296537441540055,
 0.057499404502644916,
 0.05600722231758946,
 0.05376107689959631,
 0.05488480232382926,
 0.053511396172411595,
 0.053684955313067835,
 0.05971690890445321,
 0.05960056272236183,
 0.06084461474020318,
 0.05850552702068304,
 0.05783910577795147,
 0.056274655282795974]

In [73]:
# resistance weights
volwr=[]
timewr=[]
target=vollistr
target=pd.Series(target)
ttt=pd.to_datetime(timelistr)
a=target.ewm(halflife='252 days', times=ttt).mean()

# a=target.ewm(alpha=0.95).mean()

for i in range(len(support_list)):
  vw=a[i]/sum(a)
  volwr.append(vw)
volwr

[0.05454142736309565,
 0.04447175064680757,
 0.043335806587990515,
 0.05963903314516592,
 0.05454659515551586,
 0.056043414824318225,
 0.05129843537489856,
 0.050817732238318473,
 0.049127371053400336,
 0.05002463343676718,
 0.0485372709582003,
 0.0542861969420281,
 0.053734325558502875,
 0.05253828862983923,
 0.05071151229850433,
 0.04839122493590713,
 0.046945934883084815,
 0.045039223943290414]

In [74]:
import plotly.graph_objects as go 
from datetime import datetime
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

fig = make_subplots( rows = 2, cols = 1, shared_xaxes = True,
                    vertical_spacing= 0.03, row_width = [0.2, 0.7])

fig.add_trace(go.Candlestick(x = df.index,
                            open = df['Open'],
                            high = df['High'], 
                            low = df['Low'], 
                            close = df['Close']),
                            row = 1 , col = 1) 


fig.add_trace(go.Bar(x =df.index , y = df['Volume'], showlegend = False ), 
                     row = 2, col =1)

fig.update(layout_xaxis_rangeslider_visible=False)

support_list.sort()
resistance_list.sort()

# merge lines that are close to each other
for i in range(1 , len(support_list)):
    if ( i >= len(support_list)):
        break
    if abs( support_list[i][1] - support_list[i-1][1] <= 1):
        support_list.pop(i)

for i in range(1, len(resistance_list)):
    if( i >= len(resistance_list)):
        break
    if abs( resistance_list[i][1] - resistance_list[i-1][1] <= 1):
        resistance_list.pop(i)

print(len(support_list))
print(len(resistance_list))

# gen pivotlist
pivotlist = support_list + resistance_list
pivotlist.sort()
PriceSignal = [0] *len(df)

# gen PriceSignal
for row in range(0 , len(df)):
    for pivot in pivotlist:
        # the date ofcurrent candle is on or before the pivotlist
        if ( row <= pivot[0]):
            break
        # 0 indicates support level
        if ( pivot[2] == 0 ): 
            # support level check if undercut and rally
            if (df["Low"][row] <= pivot[1] and df["Close"][row] > pivot[1]):
                PriceSignal[row] = 1 # indicate it's undercut and rally
        # 1 indicates resistance level
        elif ( pivot[2] == 1):
            # resistance level check if it rallies above the level and reverse
            if ( df["High"][row] >= pivot[1] and df["Close"][row] < pivot[1]):
                PriceSignal[row] = 2

df["PriceSignal"] = PriceSignal

# gen SupportPos & ResistPos
def SupportPos(x):
    if (x["PriceSignal"] == 1):
        return x["Low"] - 2e-3
    else:
        return np.nan

def ResistPos(x):
    if ( x["PriceSignal"] == 2):
        return x["High"] + 2e-3
    else:
        return np.nan
    
df["SupportPos"] = df.apply( lambda row: SupportPos(row), axis = 1)
df["ResistPos"] =  df.apply( lambda row: ResistPos(row), axis = 1)

# here starts the plot
counter1 = 0
counter2 = 0

while(True):
    if (counter1 > len(support_list) - 1):
        break
    fig.add_shape( type = 'line', line=dict(color="Green",width=1) ,x0 = support_list[counter1][0]-3 , y0 = support_list[counter1][1],
                  x1 = df.shape[0], y1 = support_list[counter1][1])
    
    counter1 += 1

while(True):
    if (counter2 > len(resistance_list) - 1):
          break
    fig.add_shape( type = 'line', line=dict(color="Red",width=1),x0 = resistance_list[counter2][0] - 3 , y0 = resistance_list[counter2][1], 
                x1 = df.shape[0], y1 = resistance_list[counter2][1])
    
    counter2 +=1


# add scatter points
fig.add_scatter(x = df.index , y = df['SupportPos'] ,mode = "markers",
                marker = dict(size = 8 , color = "MediumPurple"), name = "Support_Signal")

fig.add_scatter(x = df.index , y = df['ResistPos'] ,mode = "markers",
                marker = dict(size = 8 , color = "MediumBlue"), name = "Resistance_Signal")
fig.show()



11
13


In [75]:
# Backtesting Strategy
df_rtn = df.copy()

df_rtn.loc[0,"netvalue"] = 1  

for i in range(0,len(df_rtn)-1):
    if (df_rtn['PriceSignal'][i] == 2):
    
        #rt.append(max((partial_book['KF_MID'][i]-partial_book['ASK_PRICE_1'][i])/partial_book['ASK_PRICE_1'][i],0))
        df_rtn.loc[i+1, "return"] = ( - ( (df["Open"][i+1]) - (df["Close"][i+1]))/(df["Close"][i+1]) ) #short at open
        #times.append( (df["Date"][i+1]) )
        df_rtn["netvalue"][i+1] = df_rtn["netvalue"][i] * (1+df_rtn["return"][i+1])
    elif (df_rtn['PriceSignal'][i] == 1):
        df_rtn.loc[i+1, "return"] =(   ( (df["Open"][i+1]) - (df["Close"][i+1]))/(df["Close"][i+1]) ) #long at open
        
        df_rtn["netvalue"][i+1] = df_rtn["netvalue"][i] * (1+df_rtn["return"][i+1])
        #times.append( (df["Date"][i+1]) )
    else:
        
        df_rtn["netvalue"][i+1] = df_rtn["netvalue"][i]

/var/folders/mc/wrt64hkd3_x5cf174lyx50540000gn/T/ipykernel_20720/44209784.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/mc/wrt64hkd3_x5cf174lyx50540000gn/T/ipykernel_20720/44209784.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/mc/wrt64hkd3_x5cf174lyx50540000gn/T/ipykernel_20720/44209784.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/mc/wrt64hkd3_x5cf174lyx50540000gn/T/ipykernel

In [76]:
df_rtn

,Date,Open,High,Low,Close,Adj Close,Volume,PriceSignal,SupportPos,ResistPos,netvalue,return
0,2020-10-19,207.389999,207.910004,204.589996,205.690002,197.162338,3235300,0,NaN,NaN,1.000000,NaN
1,2020-10-20,206.699997,211.380005,206.470001,208.029999,199.405334,3335500,0,NaN,NaN,1.000000,NaN
2,2020-10-21,207.389999,207.949997,202.009995,202.910004,194.497589,3083600,0,NaN,NaN,1.000000,NaN
3,2020-10-22,203.199997,206.369995,202.199997,205.399994,196.884369,2136900,0,NaN,NaN,1.000000,NaN
4,2020-10-23,206.759995,208.210007,204.020004,205.039993,196.539276,2110200,0,NaN,NaN,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
500,2022-10-13,290.420013,308.760010,287.750000,307.070007,307.070007,2724900,1,287.748000,NaN,1.022132,NaN
501,2022-10-14,307.459991,311.619995,299.070007,299.989990,299.989990,2397600,0,NaN,NaN,1.047584,0.024901
502,2022-10-17,304.410004,311.619995,304.140015,306.709991,306.709991,3118600,0,NaN,NaN,1.047584,NaN
503,2022-10-18,321.670013,324.480011,312.000000,313.850006,313.850006,6428600,2,NaN,324.482011,1.047584,NaN


In [77]:
df_rtn[0:30]

,Date,Open,High,Low,Close,Adj Close,Volume,PriceSignal,SupportPos,ResistPos,netvalue,return
0,2020-10-19,207.389999,207.910004,204.589996,205.690002,197.162338,3235300,0,NaN,NaN,1.0,NaN
1,2020-10-20,206.699997,211.380005,206.470001,208.029999,199.405334,3335500,0,NaN,NaN,1.0,NaN
2,2020-10-21,207.389999,207.949997,202.009995,202.910004,194.497589,3083600,0,NaN,NaN,1.0,NaN
3,2020-10-22,203.199997,206.369995,202.199997,205.399994,196.884369,2136900,0,NaN,NaN,1.0,NaN
4,2020-10-23,206.759995,208.210007,204.020004,205.039993,196.539276,2110200,0,NaN,NaN,1.0,NaN
5,2020-10-26,202.490005,202.809998,198.509995,201.139999,192.800980,3249700,0,NaN,NaN,1.0,NaN
6,2020-10-27,200.000000,200.589996,195.410004,195.679993,187.567337,3430400,0,NaN,NaN,1.0,NaN
7,2020-10-28,191.309998,193.369995,188.940002,189.770004,181.902359,4214600,0,NaN,NaN,1.0,NaN
8,2020-10-29,189.800003,192.350006,186.899994,189.940002,182.065323,3581200,0,NaN,NaN,1.0,NaN
9,2020-10-30,189.509995,190.399994,185.520004,189.039993,181.202621,3658800,0,NaN,NaN,1.0,NaN


In [78]:
fig = make_subplots( rows = 2, cols = 1, shared_xaxes = True,
                    vertical_spacing= 0.03, row_width = [0.2, 0.7])

fig.add_trace(go.Candlestick(x = df_rtn['Date'],
                            open = df_rtn['Open'],
                            high = df_rtn['High'], 
                            low = df_rtn['Low'], 
                            close = df_rtn['Close']),
                            row = 1 , col = 1) 


fig.add_trace(go.Bar(x =df_rtn['Date'] , y = df_rtn['Volume'], showlegend = False ), 
                     row = 2, col =1)

fig.update(layout_xaxis_rangeslider_visible=False)

counter1 = 0
counter2 = 0

while(True):
    if (counter1 > len(support_list) - 1):
        break
    fig.add_shape( type = 'line', line=dict(color="Green",width=1) ,x0 = support_list[counter1][0]-3 , y0 = support_list[counter1][1],
                  x1 = df_rtn.shape[0], y1 = support_list[counter1][1])
    
    counter1 += 1

while(True):
    if (counter2 > len(resistance_list) - 1):
        break
  
    fig.add_shape( type = 'line', line=dict(color="Red",width=1),x0 = resistance_list[counter2][0] - 3 , y0 = resistance_list[counter2][1], 
                x1 = df_rtn.shape[0], y1 = resistance_list[counter2][1])
    counter2 +=1

# fig.add_scatter(x = df.index , y = df['PointPos'] ,mode = "markers",
#                 marker = dict(size = 8 , color = "MediumPurple"), name = "Signal")

fig.add_scatter(x = df_rtn['Date'] , y = df_rtn['SupportPos'] ,mode = "markers",
                marker = dict(size = 8 , color = "MediumPurple"), name = "Support_Signal")

fig.add_scatter(x = df_rtn['Date'] , y = df_rtn['ResistPos'] ,mode = "markers",
                marker = dict(size = 8 , color = "MediumBlue"), name = "Resistance_Signal")


fig.update_layout(xaxis_range=[df_rtn['Date'][0],
                               df_rtn['Date'][len(df_rtn)-1]])

fig.show()

In [79]:
fig =go.Figure()
fig.add_trace(go.Scatter(x = df_rtn['Date'] , y = df_rtn['netvalue'] ,mode = "lines",
                         marker = dict(size = 8 , color = "Black"), name = "Net Value"))
fig.show()


In [80]:
import ipynb.fs.defs.sector_research as sr
sup_df, sup_support_list, sup_resistance_list = sr.gen_sup_prced_df(stock,'2020-10-18', '2022-10-20')
sup_df_rtn = sr.gen_df_rtn(sup_df)

# overlay exp and equal tgt
fig =go.Figure()
fig.add_trace(go.Scatter(x = df_rtn['Date'] , y = df_rtn['netvalue'] ,mode = "lines",
                         marker = dict(size = 8 ), name = "EXPO Net Value"))
fig.add_trace(go.Scatter(x = sup_df_rtn['Date'] , y = sup_df_rtn['netvalue'] ,mode = "lines",
                         marker = dict(size = 8 ), name = "EQUAL Net Value"))
fig.show()

[*********************100%***********************]  1 of 1 completed


/Users/jin/Desktop/556421/test/sector_research.ipynb:313: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jin/Desktop/556421/test/sector_research.ipynb:306: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jin/Desktop/556421/test/sector_research.ipynb:313: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jin/Desktop/556421/test/sector_research.ipynb:310: SettingWithCopyWarning:


A value is trying to be set on a copy of a 